In [ ]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 11.4 MB/s eta 0:00:00


In [ ]:
import google.generativeai as genai
from google.colab import userdata
gemini_key = userdata.get('gemini-key')


genai.configure(api_key=gemini_key)
model = genai.GenerativeModel('gemini-1.5-flash')

In [ ]:
import google.generativeai as genai
import pandas as pd

import time
t=5

In [ ]:
# Configure the API key

# Initialize the model
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

# Load the CSV file into a DataFrame
file_path = '/content/output.tsv'  # Adjust the path to the uploaded file if necessary
df = pd.read_csv(file_path,sep='\t', encoding='latin1')

In [ ]:
import pandas as pd
import random
import google.generativeai as genai
import csv

# Function to extract questions and choices
def extract_question_choices(merged_text):
    parts = merged_text.split('?')
    if len(parts) < 2:
        return pd.Series([None, None])

    question = parts[0].strip() + '?'
    choices = parts[1].strip().split('\n')
    choices = [choice.strip() for choice in choices if choice.strip()]
    return pd.Series([question, choices])

# Function to get an answer from Gemini
def get_gemini_answer(prompt):
    try:
        response = genai.generate_text(model="gemini-1.5-turbo", text=prompt)
        if response.candidates:
            for candidate in response.candidates:
                if candidate.content:
                    return candidate.content.strip()
        return "Erreur: Aucun contenu valide retourné"
    except Exception as e:
        print(f"Erreur lors de la génération de la réponse : {e}")
        return "Erreur"

# Load CSV file into DataFrame
file_path = '/cti-mcq.tsv'

# Try different encodings and delimiters if needed
try:
    df = pd.read_csv(file_path,sep="\t", encoding="latin1", header=0, engine='python')
except UnicodeDecodeError:
    print("Error: 'latin1' encoding failed. Trying 'utf-8'")
    df = pd.read_csv(file_path,sep="\t", encoding="utf-8", header=0, engine='python')
except ParserError:
    print(f"Error: Parsing error. Check line 614 for unexpected characters or missing quotes.")
    df = pd.DataFrame()  # Create an empty DataFrame if parsing fails entirely

# Check if 'merged' column exists in the DataFrame
if 'merged' in df.columns:
    # Open the output CSV file
    with open('gemini_results.csv', 'w', newline='') as csvfile:
        fieldnames = ['question', 'gemini_answer']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        # Iterate over each row in the DataFrame
        for index, row in df.iterrows():
            if pd.notnull(row['merged']):
                # Combine the base prompt with the specific question from the CSV
                prompt = f"{base_prompt}\n\nQuestion: {row['merged']}"

                # Generate response from Gemini model
                try:
                    response = model.generate_content(prompt)
                    gemini_answer = response.text.strip()  # Extract and clean up the response text

                    # Write the result to the CSV file
                    writer.writerow({'question': row['merged'], 'gemini_answer': gemini_answer})

                    print(f"Question: {row['merged']}")
                    print(f"Gemini Answer: {response}\n")
                    time.sleep(5)  # Increase the delay if necessary
                except Exception as e:
                    print(f"Erreur lors de la génération de la réponse pour la question {row['merged']}: {e}")
                    writer.writerow({'question': row['merged'], 'gemini_answer': 'Erreur'})

else:
    print("La colonne 'merged' n'existe pas dans le DataFrame.")


{'question': "Which of the following mitigations involves preventing applications from running that haven't been downloaded from legitimate repositories? Audit Execution Prevention Operating System Configuration User Account Control", 'gemini_answer': 'C'}
{'question': 'Which data source is recommended for monitoring commands that may circumvent mechanisms designed to control elevation of privileges? Command File Process User Account', 'gemini_answer': 'A'}
{'question': 'What does mitigation ID M1028 suggest to prevent privilege escalation exploits on a system? Limiting privileges of cloud accounts Preventing unsigned applications from running Minimizing applications with setuid or setgid bits set Enforcing the highest UAC level', 'gemini_answer': 'C'}
{'question': 'Which process creation is an indicator of potential SYSTEM privilege escalation according to the detection section? C:\\Windows\\System32\\services.exe C:\\Windows\\System32\\cmd.exe C:\\Windows\\System32\\rundll32.exe C:\\

In [ ]:
import pandas as pd

# Load the TSV file
input_file = '/content/cti-mcq.tsv'  # Replace with your actual file name
output_file = 'output.tsv'  # Output file name

# Read the TSV file into a DataFrame
df = pd.read_csv(input_file, sep='\t',encoding="latin1")

# Merge the specified columns into one column
df['abc'] = df['Question'].astype(str) + ' ' + df['Option A'].astype(str) + ' ' + df['Option B'].astype(str) + ' ' + df['Option C'].astype(str) + ' ' + df['Option D'].astype(str)

# Save the new column to a new TSV file
df[['abc']].to_csv(output_file, sep='\t', index=False)

print(f"New file saved as {output_file}")


New file saved as output.tsv


In [ ]:
import csv
import ast

def convert_text_to_tsv(input_file, output_file):
    """Converts a text file to a TSV file.

    Args:
        input_file: Path to the input text file.
        output_file: Path to the output TSV file.
    """
    with open(input_file, 'r') as in_file, open(output_file, 'w', newline='') as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        tsv_writer.writerow(['question', 'gemini_answer'])  # Write header

        for line in in_file:
            line = line.strip()
            if line:
                try:
                    # Check if line is in the expected dictionary format
                    if line.startswith("{'question':") and 'gemini_answer' in line:
                        # Parse the line as a dictionary
                        qa_dict = ast.literal_eval(line)
                        question = qa_dict.get('question', '')
                        answer = qa_dict.get('gemini_answer', '')
                        tsv_writer.writerow([question, answer])
                except (ValueError, SyntaxError) as e:
                    print(f"Error parsing line: {line}\n{e}")

# Example usage
input_file = '/generated_outputs (2).txt'  # Update this to your actual file path
output_file = '/output.tsv'
convert_text_to_tsv(input_file, output_file)


In [ ]:
import pandas as pd

def compare_answers(file1, file2):
    """Compares answers from two TSV files and calculates the percentage of matching answers.

    Args:
        file1: Path to the first input TSV file.
        file2: Path to the second input TSV file.

    Returns:
        float: Percentage of matching answers.
    """
    # Read the TSV files into DataFrames
    df1 = pd.read_csv(file1, sep='\t', usecols=['Question', 'GT'], encoding='latin1')
    df2 = pd.read_csv(file2, sep='\t', encoding='latin1')

    # Debug prints to check the columns
    print("Columns in first file:", df1.columns)
    print("Columns in second file:", df2.columns)

    # Ensure the required columns are present
    if 'Question' not in df1.columns or 'GT' not in df1.columns:
        raise KeyError("'Question' or 'GT' column is missing in the first file.")

    if 'Question' not in df2.columns or 'GT' not in df2.columns:
        raise KeyError("'Question' or 'GT' column is missing in the second file.")

    # Merge DataFrames on the 'Question' column, keeping only common questions
    merged_df = pd.merge(df1, df2, on='Question', suffixes=('_file1', '_file2'))

    # Debug print to check the merged DataFrame
    print("Merged DataFrame head:\n", merged_df.head())

    # Add a column to compare answers
    merged_df['same_answer'] = merged_df['GT_file1'] == merged_df['GT_file2']

    # Calculate the percentage of matching answers
    total_questions = merged_df.shape[0]
    matching_answers = merged_df['same_answer'].sum()
    percentage_correct = (matching_answers / total_questions) * 100 if total_questions > 0 else 0

    return percentage_correct, merged_df

def save_comparison_results(merged_df, output_file):
    """Saves the comparison results to a TSV file.

    Args:
        merged_df: DataFrame containing the merged comparison results.
        output_file: Path to the output TSV file.
    """
    merged_df.to_csv(output_file, sep='\t', index=False)
    print(f"Comparison results saved to {output_file}")

# Example usage
file1 = '/extracted_columns.tsv'  # Update this to your actual first file path
file2 = '/modified_questions.tsv'  # Update this to your actual second file path
output_file = '/comparison_results.tsv'

percentage_correct, comparison_df = compare_answers(file1, file2)
save_comparison_results(comparison_df, output_file)

print(f"Percentage of matching answers: {percentage_correct:.2f}%")


Columns in first file: Index(['Question', 'GT'], dtype='object')
Columns in second file: Index(['Question', 'GT'], dtype='object')
Merged DataFrame head:
                                             Question GT_file1 GT_file2
0  Which of the following mitigations involves pr...        B        C
1  Which data source is recommended for monitorin...        D        A
2  What does mitigation ID M1028 suggest to preve...        C        C
3  Which process creation is an indicator of pote...        B        C
4  In a Linux environment, what is recommended to...        C        D
Comparison results saved to /comparison_results.tsv
Percentage of matching answers: 50.68%


In [ ]:
import pandas as pd

def extract_columns(input_file, output_file, columns):
    """Extract specified columns from a TSV file and save them to a new TSV file.

    Args:
        input_file: Path to the input TSV file.
        output_file: Path to the output TSV file.
        columns: List of columns to extract.
    """
    # Read the TSV file into a DataFrame
    df = pd.read_csv(input_file, sep='\t', encoding='latin1')

    # Extract the specified columns
    df_extracted = df[columns]

    # Save the extracted columns to a new TSV file
    df_extracted.to_csv(output_file, sep='\t', index=False)
    print(f"Extracted columns saved to {output_file}")

# Example usage
input_file = '/cti-mcq.tsv'  # Update this to your actual file path
output_file = '/extracted_columns.tsv'
columns_to_extract = ['Question', 'GT']

extract_columns(input_file, output_file, columns_to_extract)


Extracted columns saved to /extracted_columns.tsv


In [ ]:
import pandas as pd

def extract_text_before_question_mark(input_file, output_file):
    """Extract text before the first '?' in the 'Question' column and save to a new TSV file.

    Args:
        input_file: Path to the input TSV file.
        output_file: Path to the output TSV file.
    """
    # Read the TSV file into a DataFrame
    df = pd.read_csv(input_file, sep='\t', encoding='latin1')

    # Extract text before the first '?'
    df['Question'] = df['question'].str.split('?').str[0] + '?'

    # Save the modified DataFrame to a new TSV file
    df.to_csv(output_file, sep='\t', index=False)
    print(f"Modified questions saved to {output_file}")

# Example usage
input_file = '/output (2).tsv'  # Update this to your actual file path
output_file = '/modified_questions.tsv'

extract_text_before_question_mark(input_file, output_file)


Modified questions saved to /modified_questions.tsv
